In [ ]:
!git clone https://github.com/m6129/lag-llama.git -q
# подгрузка наборов данных
!git clone https://github.com/zhouhaoyi/ETDataset -q
!pip install --upgrade gdown -q #https://github.com/wkentaro/gdown/issues/333
!gdown --fuzzy https://drive.google.com/file/d/1nXdMIJ7K201Bx3IBGNiaNFQ6FzeDEzIr/view?usp=drive_link -q
!gdown --fuzzy https://drive.google.com/file/d/1rN79CxW3Vldp-WDuSoG0bKq9tYQR79UK/view?usp=drive_link -q
!gdown --fuzzy https://drive.google.com/file/d/1WMKg7KevVEEd9jrfLG8mcpOequZMbjlM/view?usp=drive_link -q

!unzip /content/weather.zip
!unzip /content/exchange_rate.zip
!unzip /content/illness.zip
import pandas as pd

Archive:  /content/weather.zip
   creating: weather/
  inflating: weather/weather.csv     
Archive:  /content/exchange_rate.zip
   creating: exchange_rate/
  inflating: exchange_rate/.DS_Store  
  inflating: exchange_rate/exchange_rate.csv  
Archive:  /content/illness.zip
   creating: illness/
  inflating: illness/national_illness.csv  


https://github.com/time-series-foundation-models/lag-llama/issues/35
не поддерживает ковариаты

In [ ]:
path = '/content/'

In [ ]:
ETTh1 = pd.read_csv(path + 'ETDataset/ETT-small/ETTh1.csv', index_col=0, parse_dates=True)
ETTh2 = pd.read_csv(path + 'ETDataset/ETT-small/ETTh2.csv', index_col=0, parse_dates=True)
ETTm1 = pd.read_csv(path + 'ETDataset/ETT-small/ETTm1.csv', index_col=0, parse_dates=True)
ETTm2 = pd.read_csv(path + 'ETDataset/ETT-small/ETTm2.csv', index_col=0, parse_dates=True)
exchange_rate = pd.read_csv(path + 'exchange_rate/exchange_rate.csv', index_col=0, parse_dates=True)
illness = pd.read_csv(path + 'illness/national_illness.csv', index_col=0, parse_dates=True)
weather = pd.read_csv(path + 'weather/weather.csv', index_col=0, parse_dates=True)

In [ ]:
cd lag-llama

/content/lag-llama


In [ ]:
!pip install -r requirements.txt --quiet # this could take some time # ignore the errors displayed by colab
!huggingface-cli download time-series-foundation-models/Lag-Llama lag-llama.ckpt --local-dir /content/lag-llama
from itertools import islice

from matplotlib import pyplot as plt
import matplotlib.dates as mdates

import torch
from gluonts.evaluation import make_evaluation_predictions, Evaluator
from gluonts.dataset.repository.datasets import get_dataset

from gluonts.dataset.pandas import PandasDataset

from lag_llama.gluon.estimator import LagLlamaEstimator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 74.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 87.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 778.1/778.1 kB 58.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.8/266.8 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 64.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. Th

/usr/local/lib/python3.10/dist-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
def get_lag_llama_predictions(dataset, prediction_length, device, context_length=32, use_rope_scaling=False, num_samples=100):
    ckpt = torch.load("lag-llama.ckpt", map_location=device) # Uses GPU since in this Colab we use a GPU.
    estimator_args = ckpt["hyper_parameters"]["model_kwargs"]

    rope_scaling_arguments = {
        "type": "linear",
        "factor": max(1.0, (context_length + prediction_length) / estimator_args["context_length"]),
    }

    estimator = LagLlamaEstimator(
        ckpt_path="lag-llama.ckpt",
        prediction_length=prediction_length,
        context_length=context_length, # Lag-Llama was trained with a context length of 32, but can work with any context length

        # estimator args
        input_size=estimator_args["input_size"],
        n_layer=estimator_args["n_layer"],
        n_embd_per_head=estimator_args["n_embd_per_head"],
        n_head=estimator_args["n_head"],
        scaling=estimator_args["scaling"],
        time_feat=estimator_args["time_feat"],
        rope_scaling=rope_scaling_arguments if use_rope_scaling else None,

        batch_size=1,
        num_parallel_samples=100,
        device=device,
    )

    ckpt = torch.load("lag-llama.ckpt", map_location=device)
    lightning_module = estimator.create_lightning_module()
    transformation = estimator.create_transformation()
    predictor = estimator.create_predictor(transformation, lightning_module)

    forecast_it, ts_it = make_evaluation_predictions(
        dataset=dataset,
        predictor=predictor,
        num_samples=num_samples
    )
    forecasts = list(forecast_it)
    tss = list(ts_it)

    return forecasts, tss

In [ ]:
import os
import random
import numpy as np
import torch


def set_seed(seed, deterministic=True):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = deterministic
        torch.backends.cudnn.benchmark = False
    os.environ["PYTHONHASHSEED"] = str(seed)

In [ ]:
def prep_lag_func(df, target):
  for col in df.columns:
    # если столбец не является строковым типом
    if df[col].dtype != 'object' and pd.api.types.is_string_dtype(df[col]) == False:
        df[col] = df[col].astype('float32')
  dataset = PandasDataset(df, target=target)
  return dataset
device = torch.device("cuda:0")
prediction_length = 96  # Определите длину прогноза.
num_samples = 400 # количество выборок из вероятностного распределения для каждого временного интервала

## ETTh1

In [ ]:
%%time
set_seed(42)
step_forecasting = [24, 96, 192, 336, 720]
backtest_dataset = prep_lag_func(ETTh1, "OT")
for step in step_forecasting:
  prediction_length=step
  forecasts, tss = get_lag_llama_predictions(backtest_dataset, prediction_length, device, num_samples) # tss временной ряд (для визуализации)
  evaluator = Evaluator()
  agg_metrics, ts_metrics = evaluator(iter(tss), iter(forecasts))
  mse = agg_metrics['MSE']
  print(f'горизонт прогнозирования: {step}, mse: {mse}')

Running evaluation: 1it [00:00, 17.16it/s]


горизонт прогнозирования: 24, mse: 0.24983404080073038


Running evaluation: 1it [00:00, 20.27it/s]


горизонт прогнозирования: 96, mse: 1.6669683456420898


Running evaluation: 1it [00:00, 39.69it/s]


горизонт прогнозирования: 192, mse: 4.634436289469401


Running evaluation: 1it [00:00, 36.01it/s]


горизонт прогнозирования: 336, mse: 3.0204364231654575


Running evaluation: 1it [00:00, 34.60it/s]


горизонт прогнозирования: 720, mse: 2.921006266276042
CPU times: user 6min 51s, sys: 1.43 s, total: 6min 52s
Wall time: 7min 4s


In [ ]:
%%time
set_seed(42)
step_forecasting = [24, 96, 192, 336, 720]
backtest_dataset = prep_lag_func(ETTh1[['OT']], "OT") # провекра на одномерное прогнозирование
for step in step_forecasting:
  prediction_length=step
  forecasts, tss = get_lag_llama_predictions(backtest_dataset, prediction_length, device, num_samples)
  evaluator = Evaluator()
  agg_metrics, ts_metrics = evaluator(iter(tss), iter(forecasts))
  mse = agg_metrics['MSE']
  print(f'горизонт прогнозирования: {step}, mse: {mse}')

<ipython-input-8-4eb7658b6463>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype('float32')
Running evaluation: 1it [00:00, 32.71it/s]


горизонт прогнозирования: 24, mse: 0.24983404080073038


Running evaluation: 1it [00:00, 33.14it/s]


горизонт прогнозирования: 96, mse: 1.6669683456420898


Running evaluation: 1it [00:00, 33.87it/s]


горизонт прогнозирования: 192, mse: 4.634436289469401


Running evaluation: 1it [00:00, 30.32it/s]


горизонт прогнозирования: 336, mse: 3.0204364231654575


Running evaluation: 1it [00:00, 35.75it/s]


горизонт прогнозирования: 720, mse: 2.921006266276042
CPU times: user 6min 44s, sys: 1.3 s, total: 6min 45s
Wall time: 6min 51s


## ETTh2

In [ ]:
%%time
set_seed(42)
step_forecasting = [24, 96, 192, 336, 720]
backtest_dataset = prep_lag_func(ETTh2, "OT")
for step in step_forecasting:
  prediction_length=step
  forecasts, tss = get_lag_llama_predictions(backtest_dataset, prediction_length, device, num_samples) # tss временной ряд (для визуализации)
  evaluator = Evaluator()
  agg_metrics, ts_metrics = evaluator(iter(tss), iter(forecasts))
  mse = agg_metrics['MSE']
  print(f'горизонт прогнозирования: {step}, mse: {mse}')

Running evaluation: 1it [00:00, 35.01it/s]


горизонт прогнозирования: 24, mse: 36.71279652913412


Running evaluation: 1it [00:00, 33.45it/s]


горизонт прогнозирования: 96, mse: 31.20037333170573


Running evaluation: 1it [00:00, 37.00it/s]


горизонт прогнозирования: 192, mse: 71.92296346028645


Running evaluation: 1it [00:00, 36.73it/s]


горизонт прогнозирования: 336, mse: 22.438037690662203


Running evaluation: 1it [00:00, 30.67it/s]


горизонт прогнозирования: 720, mse: 65.13684895833333
CPU times: user 6min 57s, sys: 553 ms, total: 6min 58s
Wall time: 7min 2s


## ETTm1

In [ ]:
%%time
set_seed(42)
step_forecasting = [24, 96, 192, 336, 720]
backtest_dataset = prep_lag_func(ETTm1, "OT")
for step in step_forecasting:
  prediction_length=step
  forecasts, tss = get_lag_llama_predictions(backtest_dataset, prediction_length, device, num_samples) # tss временной ряд (для визуализации)
  evaluator = Evaluator()
  agg_metrics, ts_metrics = evaluator(iter(tss), iter(forecasts))
  mse = agg_metrics['MSE']
  print(f'горизонт прогнозирования: {step}, mse: {mse}')

Running evaluation: 1it [00:00, 32.05it/s]


горизонт прогнозирования: 24, mse: 3.711392084757487


Running evaluation: 1it [00:00, 46.87it/s]


горизонт прогнозирования: 96, mse: 1.4289960861206055


Running evaluation: 1it [00:00, 45.05it/s]


горизонт прогнозирования: 192, mse: 6.824401219685872


Running evaluation: 1it [00:00, 36.77it/s]


горизонт прогнозирования: 336, mse: 10.272527785528274


Running evaluation: 1it [00:00, 40.98it/s]


горизонт прогнозирования: 720, mse: 5.618724229600694
CPU times: user 6min 55s, sys: 606 ms, total: 6min 55s
Wall time: 7min 4s


## ETTm2

In [ ]:
%%time
set_seed(42)
step_forecasting = [24, 96, 192, 336, 720]
backtest_dataset = prep_lag_func(ETTm2, "OT")
for step in step_forecasting:
  prediction_length=step
  forecasts, tss = get_lag_llama_predictions(backtest_dataset, prediction_length, device, num_samples) # tss временной ряд (для визуализации)
  evaluator = Evaluator()
  agg_metrics, ts_metrics = evaluator(iter(tss), iter(forecasts))
  mse = agg_metrics['MSE']
  print(f'горизонт прогнозирования: {step}, mse: {mse}')

Running evaluation: 1it [00:00, 36.99it/s]


горизонт прогнозирования: 24, mse: 7.952391306559245


Running evaluation: 1it [00:00, 43.02it/s]


горизонт прогнозирования: 96, mse: 66.84247334798177


Running evaluation: 1it [00:00, 44.00it/s]


горизонт прогнозирования: 192, mse: 89.42974853515625


Running evaluation: 1it [00:00, 40.34it/s]


горизонт прогнозирования: 336, mse: 177.1654808407738


Running evaluation: 1it [00:00, 62.37it/s]


горизонт прогнозирования: 720, mse: 49.69803059895833
CPU times: user 7min, sys: 676 ms, total: 7min
Wall time: 7min 7s


## exchange_rate

In [ ]:
%%time
set_seed(42)
step_forecasting = [24, 96, 192, 336, 720]
backtest_dataset = prep_lag_func(exchange_rate, "OT")
for step in step_forecasting:
  prediction_length=step
  forecasts, tss = get_lag_llama_predictions(backtest_dataset, prediction_length, device, num_samples) # tss временной ряд (для визуализации)
  evaluator = Evaluator()
  agg_metrics, ts_metrics = evaluator(iter(tss), iter(forecasts))
  mse = agg_metrics['MSE']
  print(f'горизонт прогнозирования: {step}, mse: {mse}')

Running evaluation: 1it [00:00, 35.52it/s]


горизонт прогнозирования: 24, mse: 0.00040595113144566614


Running evaluation: 1it [00:00, 37.09it/s]


горизонт прогнозирования: 96, mse: 0.00018497119890525937


Running evaluation: 1it [00:00, 40.91it/s]


горизонт прогнозирования: 192, mse: 0.00213422579690814


Running evaluation: 1it [00:00, 34.24it/s]


горизонт прогнозирования: 336, mse: 0.0021996180571260907


Running evaluation: 1it [00:00, 32.72it/s]


горизонт прогнозирования: 720, mse: 0.011285286479526096
CPU times: user 7min 1s, sys: 564 ms, total: 7min 1s
Wall time: 7min 6s


In [ ]:
exchange_rate

,0,1,2,3,4,5,6,OT
date,,,,,,,,
1990-01-01,0.785500,1.611000,0.861698,0.634196,0.211242,0.006838,0.525486,0.593000
1990-01-02,0.781800,1.610000,0.861104,0.633513,0.211242,0.006863,0.523972,0.594000
1990-01-03,0.786700,1.629300,0.861030,0.648508,0.211242,0.006975,0.526316,0.597300
1990-01-04,0.786000,1.637000,0.862069,0.650618,0.211242,0.006953,0.523834,0.597000
1990-01-05,0.784900,1.653000,0.861995,0.656254,0.211242,0.006940,0.527426,0.598500
...,...,...,...,...,...,...,...,...
2010-10-06,0.718494,1.222195,0.737485,0.969974,0.143697,0.008500,0.688565,0.690846
2010-10-07,0.721839,1.223459,0.741155,0.977297,0.143763,0.008595,0.690288,0.695701
2010-10-08,0.723197,1.234111,0.745184,0.984446,0.143997,0.008562,0.691419,0.695943


## illness

In [ ]:
%%time
set_seed(42)
step_forecasting = [24, 36, 48, 60]
backtest_dataset = prep_lag_func(illness, "OT")
for step in step_forecasting:
  prediction_length=step
  forecasts, tss = get_lag_llama_predictions(backtest_dataset, prediction_length, device, num_samples) # tss временной ряд (для визуализации)
  evaluator = Evaluator()
  agg_metrics, ts_metrics = evaluator(iter(tss), iter(forecasts))
  mse = agg_metrics['MSE']
  print(f'горизонт прогнозирования: {step}, mse: {mse}')

Running evaluation: 1it [00:00, 35.35it/s]


горизонт прогнозирования: 24, mse: 66313322496.0


Running evaluation: 1it [00:00, 38.43it/s]


горизонт прогнозирования: 36, mse: 148014308465.77777


Running evaluation: 1it [00:00, 36.12it/s]


горизонт прогнозирования: 48, mse: 164027432960.0


Running evaluation: 1it [00:00, 34.73it/s]


горизонт прогнозирования: 60, mse: 102364479488.0
CPU times: user 27.9 s, sys: 177 ms, total: 28.1 s
Wall time: 30.7 s


**Стоит отметить, что наборы данных ETTh1, ETTh2, ETTm1 использовались для обучения модел**и.

При дообучении модели время на прогнозирование увеличивается, а точность прогноза значительно уменьшается
